<a href="https://colab.research.google.com/github/hhhhhenanZ/2025MoMo_Workshop_Demo/blob/main/2025MoMo_Workshop_Demo_User_Guide_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2025MoMo Workshop Demo — Colab User Guide

This notebook runs your full workflow **directly inside your Google Drive folder** after mounting Drive.

### What this version does
1. Mounts Google Drive.
2. You set `DRIVE_FOLDER_PATH` to the folder path in Drive (e.g., `/content/drive/MyDrive/2025MoMo_Workshop_Demo` or a shortcut to your shared folder).
3. Runs steps 1–6 **in place** inside that folder:
   - **Step 1:** `Read_Zone_Data.py`
   - **Step 2:** `Read_OSM_Driving.py`
   - **Step 3:** `connector_generation_driving.py` → create `connected_network/`, rename/move outputs
   - **Step 4:** Validation by running `Network_Validator_Main.py` **after commenting out** lines referencing `GMNS_Plus_Readiness_Validator.py` and `GMNS_Tools`
   - **Step 5:** Copy `DTALite_Test.py` into `connected_network/` and run it
   - **Step 6:** Zip final results (zip will be saved into the same Drive folder)


In [1]:
#@title 🔐 Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
print('Drive mounted at /content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive mounted at /content/drive


In [2]:
#@title 📦 Install required packages
!pip install -q osm2gmns==1.0.1
!pip install -q "geopandas[all]"
!pip install -q DTALite
print("✅ Packages installed: osm2gmns, geopandas, DTALite")

✅ Packages installed: osm2gmns, geopandas, DTALite


In [10]:
#@title 🗂️ Configure shared folder (by link ID) and local working directory

# Shared folder link provided:
SHARED_FOLDER_ID = "1nIcNlDKgTV5OtNKHZMmsaA8C951Cu2wg"
LOCAL_ROOT = "/content/2025MoMo_Workshop_Demo"  # local working directory

import os, subprocess, shlex

# Make sure local dir exists
os.makedirs(LOCAL_ROOT, exist_ok=True)

# Download entire shared folder into LOCAL_ROOT
cmd = f"gdown --folder https://drive.google.com/drive/folders/{SHARED_FOLDER_ID} -O {LOCAL_ROOT}"
print("Downloading folder:", cmd)
proc = subprocess.run(shlex.split(cmd), capture_output=True, text=True)
print(proc.stdout)
print(proc.stderr)
if proc.returncode != 0:
    raise RuntimeError("gdown folder download failed — check that the folder is accessible.")

print("Working in local copy of shared folder:", LOCAL_ROOT)


Retrieving folder 1YNQzNu9_uRY57uS_rZQntnRsG1VdEjdq data
Processing file 12jDAPfV0fwyRfKUd0ZDKX3Ff-aHfa6o3 San Francisco.osm
Processing file 1aNI3ludARqabBWnJ5vp028YrG9T6u16d TAZ_SF.cpg
Processing file 1pbMzT6qS_ZgNLLDExQz6UEGAv095Vf2X TAZ_SF.dbf
Processing file 1u35eYWk2YJbOMMM5rfcm9TLpU_AeeUCq TAZ_SF.prj
Processing file 1mObxhrcPFPOyvecWqM76-jG5YM0C_ZV3 TAZ_SF.qmd
Processing file 1o792F9kvwufSanOlUcaAe1izVHOH3dEw TAZ_SF.shp
Processing file 170I074FgdfcfEyUNKCLlSN5JRH35vj_x TAZ_SF.shx
Retrieving folder 18VdOt09Q0p_REjcbOGm0_2k9wBHHPDwO GMNS_Tools
Retrieving folder 1I-U-wi74HBTK6d3J7mxBmR7u5F-7ISUz Accessibility_checking_tools
Processing file 1iGUd0Eus9S6epaWsN6fvQBnAJSwvImFK NEXTA.exe
Processing file 1lagDgafowEAshoyhovBA7rBvNGdIJaNm settings.csv
Processing file 1snXHMJW6yCflpPUP4hYJNUr13nZIqjH0 TAPLite_0515_2025.exe
Processing file 1peXmuT7rLo8BHL_LFlgV3RPawDBBgJgr connector_generation_driving.py
Processing file 1Nqyd4NCUpnBuEpWud0Ac4mI-KFVTopLZ demand.csv
Processing file 1k_Ut6uzwqk

## 🔧 Run Pipeline (in-place inside your Drive folder)
All scripts are expected to be located in the Drive folder set above. Outputs will be written back to the same folder.


In [11]:
#@title ▶️ Step 1 — Run Read_Zone_Data.py
import os, subprocess, sys

# Switch into the local copy we downloaded with gdown
os.chdir(LOCAL_ROOT)
print("CWD:", os.getcwd())

script = "Read_Zone_Data.py"
if not os.path.exists(script):
    raise FileNotFoundError(f"Missing {script} in {os.getcwd()}")

# Run the script (it will directly use shapefiles in this folder)
res = subprocess.run([sys.executable, script], capture_output=True, text=True)

print('--- STDOUT ---\n', res.stdout)
print('--- STDERR ---\n', res.stderr)

if res.returncode != 0:
    raise RuntimeError("Step 1 failed")

print("Step 1 complete.")


CWD: /content/2025MoMo_Workshop_Demo
--- STDOUT ---
 Shapefile loaded successfully.
Current CRS: EPSG:4326
Available columns: Index(['taz1454', 'district', 'county', 'gacres', 'geometry'], dtype='object')
Detected zone identifier column: 'taz1454'
Sample values: [1, 3, 5]
Total zones: 196

SUMMARY STATISTICS
Total number of zones: 196
Zone ID column: taz1454
CRS: EPSG:4326
Centroid coordinates calculated and boundaries preserved.
Figure(2000x800)
Centroid and boundary data saved to zone_centroid_and_boundary.csv

File created: zone_centroid_and_boundary.csv

GeoDataFrame now has two geometries:
- 'geometry' column: Point geometries (centroids) from x_coord, y_coord
- 'boundary_geometry' column: Original polygon boundaries

For subsequent steps, you can:
- Use gdf['geometry'] for centroid-based operations
- Use gdf['boundary_geometry'] for boundary-based operations
- Access both from the CSV via 'boundary_geometry' column (WKT format)

--- STDERR ---
 
Step 1 complete.


In [12]:
#@title ▶️ Step 2 — Run Read_OSM_Driving.py
import os, subprocess, sys

# Ensure we are in the shared project folder
os.chdir(LOCAL_ROOT)
print("CWD:", os.getcwd())

script = "Read_OSM_Driving.py"
if not os.path.exists(script):
    raise FileNotFoundError(f"Missing {script} in {os.getcwd()}")

# Run the script
res = subprocess.run([sys.executable, script], capture_output=True, text=True)

print('--- STDOUT ---\n', res.stdout)
print('--- STDERR ---\n', res.stderr)

if res.returncode != 0:
    raise RuntimeError("Step 2 failed")

print("Step 2 complete.")


CWD: /content/2025MoMo_Workshop_Demo
--- STDOUT ---
 
--- STDERR ---
 I0914 07:32:25.811787   11826 functions.cpp:43] loading data from osm file
I0914 07:32:25.813334   11826 osmnetwork.cpp:570] no valid boundary information in the osm file
I0914 07:32:36.431709   11826 osmnetwork.cpp:601] nodes: 1707634 ways: 179131 relations: 1000
W0914 07:32:36.703166   11826 osmnetwork.cpp:516] unkown way member 289548836 in relation 3846332, the relation will not be imported
W0914 07:32:36.703291   11826 osmnetwork.cpp:516] unkown way member 636748504 in relation 8838432, the relation will not be imported
I0914 07:32:36.930367   11826 functions.cpp:46] start to build network
I0914 07:32:38.354205   11826 functions.cpp:48] build network done
I0914 07:32:38.356759   11826 networks.cpp:471] 197 intersections consolidated
I0914 07:32:38.361884   11826 networks.cpp:380] Node activity info generated
I0914 07:32:38.361940   11826 io.cpp:91] writing network to csv files
I0914 07:32:40.265284   11826 io.cp

In [13]:
#@title ▶️ Step 3 — Run connector_generation_driving.py
import os, shutil, subprocess, sys

# Work inside shared project folder
os.chdir(LOCAL_ROOT)
print("CWD:", os.getcwd())

script = "connector_generation_driving.py"
if not os.path.exists(script):
    raise FileNotFoundError(f"Missing {script} in {os.getcwd()}")

# Run the script
res = subprocess.run([sys.executable, script], capture_output=True, text=True)
print('--- STDOUT ---\n', res.stdout)
print('--- STDERR ---\n', res.stderr)
if res.returncode != 0:
    raise RuntimeError("Step 3 failed")

# Ensure connected_network exists
cn_dir = os.path.join(os.getcwd(), "connected_network")
os.makedirs(cn_dir, exist_ok=True)

def maybe_move(src, dst):
    if os.path.exists(src):
        os.makedirs(os.path.dirname(dst), exist_ok=True)
        shutil.move(src, dst)
        print("Moved:", src, "→", dst)
        return True
    return False

def maybe_rename_in_place(path, new_name):
    """Rename file within its current folder."""
    if os.path.exists(path):
        dst = os.path.join(os.path.dirname(path), new_name)
        if os.path.abspath(path) != os.path.abspath(dst):
            os.replace(path, dst)
            print("Renamed:", path, "→", dst)
        else:
            print("Already named:", path)
        return True
    return False

# Handle node_updated.csv → node.csv
# 1) If script output is in connected_network/, rename there
if not maybe_rename_in_place(os.path.join(cn_dir, "node_updated.csv"), "node.csv"):
    # 2) Else, if it exists in project root, move+rename into connected_network/
    if not maybe_move("node_updated.csv", os.path.join(cn_dir, "node.csv")):
        print("WARNING: node_updated.csv not found in root or connected_network")

# Handle link_updated.csv → link.csv
if not maybe_rename_in_place(os.path.join(cn_dir, "link_updated.csv"), "link.csv"):
    if not maybe_move("link_updated.csv", os.path.join(cn_dir, "link.csv")):
        print("WARNING: link_updated.csv not found in root or connected_network")

# Move demand.csv into connected_network (but only if it's not already there)
root_demand = os.path.join(os.getcwd(), "demand.csv")
cn_demand   = os.path.join(cn_dir, "demand.csv")
if os.path.exists(cn_demand):
    print("demand.csv already in connected_network/")
elif os.path.exists(root_demand):
    maybe_move(root_demand, cn_demand)
else:
    print("WARNING: demand.csv not found in project root or connected_network")

print("\n✅ Step 3 complete. Contents of connected_network/:")
print(os.listdir(cn_dir))


CWD: /content/2025MoMo_Workshop_Demo
--- STDOUT ---
 Starting to process node data...
Finding the maximum node_id in node_taz_df...
Maximum node_id in node_taz_df: 196
Adding new_node_id to node_df...
New node_id generation completed.
Filtering rows in node_df where 'zone_id' is not null...
Filtered 167 rows with non-null zone_id.
Filtered 6712 rows with null zone_id.
Saving activity_node_df to '/content/2025MoMo_Workshop_Demo/connected_network/activity_node.csv'...
File saved successfully to '/content/2025MoMo_Workshop_Demo/connected_network/activity_node.csv'.
Saving common_node_df to '/content/2025MoMo_Workshop_Demo/connected_network/common_node.csv'...
File saved successfully to '/content/2025MoMo_Workshop_Demo/connected_network/common_node.csv'.
Updating link_df with new node ID mappings...
link_df mapping completed. Original DataFrame was not modified.
Step 1: Connecting activity nodes to their zone centroids...
Step 1 complete. Activity nodes connected to zone centroids.
Step 3:

In [17]:
#@title ▶️ Step 4 — Validation
import os, re, subprocess, sys, shutil

os.chdir(LOCAL_ROOT)
print("CWD:", os.getcwd())

val_script = "Network_Validator_Main.py"
if not os.path.exists(val_script):
    raise FileNotFoundError(f"Missing {val_script} in {os.getcwd()}")

cn_dir = os.path.join(LOCAL_ROOT, "connected_network")
node_csv = os.path.join(cn_dir, "node.csv")
link_csv = os.path.join(cn_dir, "link.csv")
dmd_csv  = os.path.join(cn_dir, "demand.csv")
conn_csv = os.path.join(cn_dir, "connector_links.csv")

# 1) Prefer link.csv. If the validator insists on 'connector_links.csv',
#    ensure that filename points to link.csv (symlink if possible, else copy).
if os.path.exists(link_csv):
    if os.path.islink(conn_csv) or os.path.exists(conn_csv):
        try:
            # If it's an existing file/symlink, remove it so we can recreate as alias to link.csv
            os.remove(conn_csv)
        except Exception as e:
            print("Could not remove existing connector_links.csv:", e)
    try:
        os.symlink(link_csv, conn_csv)
        print("Created symlink:", conn_csv, "→", link_csv)
    except OSError:
        # Symlinks may be disallowed; fall back to hard copy
        shutil.copy2(link_csv, conn_csv)
        print("Symlink not available; copied:", link_csv, "→", conn_csv)
else:
    print("❗ link.csv not found at:", link_csv)

# 2) Patch obvious literals in Network_Validator_Main.py to prefer 'link.csv'
with open(val_script, "r", encoding="utf-8", errors="ignore") as f:
    src = f.read()

patched = src
patched = re.sub(r'connected_network/connector_links\.csv', 'connected_network/link.csv', patched)
patched = re.sub(r'connector_links\.csv', 'link.csv', patched)

if patched != src:
    with open(val_script, "w", encoding="utf-8") as f:
        f.write(patched)
    print("Patched validator to reference link.csv instead of connector_links.csv")

# 3) If GMNS files are present, ensure they can be imported (uncommented)
gmns_validator_py = os.path.exists("GMNS_Plus_Readiness_Validator.py")
gmns_tools_py     = os.path.exists("GMNS_Tools.py")

def uncomment_imports(path, needles):
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        lines = f.readlines()
    changed = False
    out = []
    for line in lines:
        stripped = line.lstrip()
        if stripped.startswith("#") and any(n in stripped for n in needles):
            out.append(re.sub(r'^\s*#\s?', '', line))
            changed = True
        else:
            out.append(line)
    if changed:
        with open(path, "w", encoding="utf-8") as f:
            f.writelines(out)
        print("Uncommented GMNS imports in", path)

if gmns_validator_py or gmns_tools_py:
    uncomment_imports(val_script, ["GMNS_Plus_Readiness_Validator", "GMNS_Tools"])
    if LOCAL_ROOT not in sys.path:
        sys.path.insert(0, LOCAL_ROOT)

# 4) Sanity print
for fp in [node_csv, link_csv, dmd_csv, conn_csv]:
    print(("✅ Found " if os.path.exists(fp) else "❗ Missing "), fp)

# 5) Run validator
res = subprocess.run([sys.executable, val_script], capture_output=True, text=True)
print('--- STDOUT ---\n', res.stdout)
print('--- STDERR ---\n', res.stderr)

if res.returncode != 0:
    raise RuntimeError("Validation step failed; see logs above.")
print("Step 4 complete (validator executed; link.csv data is used).")


CWD: /content/2025MoMo_Workshop_Demo
Created symlink: /content/2025MoMo_Workshop_Demo/connected_network/connector_links.csv → /content/2025MoMo_Workshop_Demo/connected_network/link.csv
✅ Found  /content/2025MoMo_Workshop_Demo/connected_network/node.csv
✅ Found  /content/2025MoMo_Workshop_Demo/connected_network/link.csv
✅ Found  /content/2025MoMo_Workshop_Demo/connected_network/demand.csv
✅ Found  /content/2025MoMo_Workshop_Demo/connected_network/connector_links.csv
--- STDOUT ---
 sample_settings.csv file created successfully!
sample_mode_type.csv file created successfully!
number_of_modes = 1
# of nodes= 7075, largest zone id (# of zones) = 196, First Through Node ID = 197, number of links = 11595
total_base_link_volume = 0.000000
read demand file demand.csv
o_zone_id: 1, d_zone_id: 1, volume: 53.0000
o_zone_id: 1, d_zone_id: 2, volume: 33.0000
o_zone_id: 1, d_zone_id: 3, volume: 59.0000
o_zone_id: 1, d_zone_id: 4, volume: 28.0000
 mode type = auto, total_volume = 149873.000000
Memmor

In [18]:
#@title ▶️ Step 5 — Copy DTALite_Test.py into connected_network and run it
import os, shutil, subprocess, sys
cn_dir = os.path.join(os.getcwd(), 'connected_network')
os.makedirs(cn_dir, exist_ok=True)

src = 'DTALite_Test.py'
dst = os.path.join(cn_dir, 'DTALite_Test.py')
if os.path.exists(src):
    shutil.copy2(src, dst)
    print('Copied DTALite_Test.py → connected_network/')
else:
    print('WARNING: DTALite_Test.py not found in project root; skipping copy.')

if os.path.exists(dst):
    res = subprocess.run([sys.executable, dst], cwd=cn_dir, capture_output=True, text=True)
    print('--- STDOUT ---\n', res.stdout)
    print('--- STDERR ---\n', res.stderr)
    if res.returncode != 0:
        print('WARNING: DTALite_Test.py exited with non-zero status. Review logs above.')
    else:
        print('Step 5 complete.')
else:
    print('Step 5 skipped (DTALite_Test.py missing).')


Copied DTALite_Test.py → connected_network/
--- STDOUT ---
 sample_settings.csv file created successfully!
sample_mode_type.csv file created successfully!
number_of_modes = 1
# of nodes= 7075, largest zone id (# of zones) = 196, First Through Node ID = 197, number of links = 11595
total_base_link_volume = 0.000000
read demand file demand.csv
o_zone_id: 1, d_zone_id: 1, volume: 53.0000
o_zone_id: 1, d_zone_id: 2, volume: 33.0000
o_zone_id: 1, d_zone_id: 3, volume: 59.0000
o_zone_id: 1, d_zone_id: 4, volume: 28.0000
 mode type = auto, total_volume = 149873.000000
Memmory creation time for 5D link path matrix: 0 hours 0 minutes 0 seconds 8 ms
The list of zero-volume zones:51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143

In [21]:
#@title 🗜️ Step 6 — Zip final results inside the project folder
import os, shutil, time

os.chdir(LOCAL_ROOT)
print("CWD:", os.getcwd())

stamp = time.strftime('%Y%m%d')
zip_name = f'connected_network_{stamp}.zip'

cn_dir = os.path.join(LOCAL_ROOT, "connected_network")
zip_path = os.path.join(LOCAL_ROOT, zip_name)

if os.path.isdir(cn_dir):
    # Create the zip inside LOCAL_ROOT
    shutil.make_archive(base_name=zip_path[:-4], format='zip', root_dir=cn_dir)
    print("✅ Created ZIP at:", zip_path)
else:
    print("❗ connected_network folder not found; nothing to zip.")


CWD: /content/2025MoMo_Workshop_Demo
✅ Created ZIP at: /content/2025MoMo_Workshop_Demo/connected_network_20250914.zip


In [22]:
#@title 🧹 Full cleanup — remove ALL local files and folders
import shutil, os

if os.path.isdir(LOCAL_ROOT):
    shutil.rmtree(LOCAL_ROOT)
    print(f"Removed entire folder: {LOCAL_ROOT}")
else:
    print("LOCAL_ROOT does not exist — nothing to remove.")

# Optionally, also clear /content completely (be careful!)
# shutil.rmtree("/content", ignore_errors=True)
# os.makedirs("/content", exist_ok=True)

print("✅ Full cleanup done. You can now re-run the download cell to fetch a fresh copy from Drive.")


Removed entire folder: /content/2025MoMo_Workshop_Demo
✅ Full cleanup done. You can now re-run the download cell to fetch a fresh copy from Drive.
